# 상호운용성

(introduction-interoperability-key-takeaway-1)=
(introduction-interoperability-key-takeaway-4)=
## 동기 부여

{ref}`분석 프레임워크 및 도구 장 <introduction:analysis-frameworks>`에서 논의한 바와 같이, 단일 세포 분석을 위한 세 가지 주요 생태계가 있습니다: R의 [Bioconductor](https://bioconductor.org/) 및 [Seurat](https://satijalab.org/seurat/index.html) 생태계와 Python 기반의 [scverse](https://scverse.org/) 생태계입니다.
신규 분석가들이 흔히 묻는 질문은 어떤 생태계를 중점적으로 배우고 사용해야 하는가입니다.
처음에는 하나에 집중하는 것이 합리적이고, 어떤 생태계에서든 성공적인 표준 분석을 수행할 수 있지만, 우리는 유능한 분석가라면 세 가지 생태계 모두에 익숙하고 그 사이를 편안하게 이동할 수 있어야 한다는 생각을 권장합니다.
이러한 접근 방식을 통해 분석가들은 구현 방식에 상관없이 최고의 성능을 내는 도구와 방법을 사용할 수 있습니다.
분석가들이 생태계 간 이동에 익숙하지 않을 때, 다른 생태계의 패키지에 비해 단점이 있는 것으로 나타났음에도 불구하고 접근하기 쉬운 패키지를 사용하는 경향이 있습니다.
분석가들이 생태계 간에 이동할 수 있는 능력은 개발자들이 프로그래밍 언어의 다양한 강점을 활용할 수 있게 해줍니다.
예를 들어, R은 복잡한 통계 모델링을 위한 강력한 내장 지원을 제공하는 반면, 대부분의 딥러닝 라이브러리는 Python에 중점을 둡니다.
공통 디스크 기반 데이터 형식과 인메모리 데이터 구조를 지원함으로써 개발자들은 분석가들이 자신의 패키지에 접근하고 자신의 방법에 가장 적합한 플랫폼을 사용할 수 있다고 확신할 수 있습니다.
여러 생태계에 익숙해지는 또 다른 동기는 데이터, 결과 및 문서의 접근성과 가용성입니다.
데이터나 결과는 종종 하나의 형식으로만 제공되므로 분석가들은 해당 형식에 익숙해야 접근할 수 있습니다.
어떤 방법을 사용할지 결정할 때 패키지 문서와 튜토리얼을 이해하기 위해서는 다른 생태계에 대한 기본적인 이해도 필요합니다.

우리는 분석가들이 모든 주요 생태계에 익숙해지도록 권장하지만, 그들 사이를 이동하는 것은 상호 운용이 가능할 때만 가능합니다.
다행히도 이 분야에서 많은 작업이 이루어졌으며, 이제 대부분의 경우 표준 패키지를 사용하는 것이 비교적 간단합니다.
이 장에서는 디스크나 메모리를 통해 생태계 간에 데이터를 이동하는 다양한 방법, 그 차이점, 그리고 장점에 대해 논의합니다.
우리는 단일 모달리티 데이터와 R과 Python 간의 이동에 중점을 두는데, 이것이 가장 일반적인 경우이기 때문입니다. 하지만 다중 모달리티 데이터와 다른 언어에 대해서도 간략히 다룹니다.

이 장을 진행하기 전에 먼저 필요한 모든 Python 패키지를 가져옵니다.

In [2]:
import tempfile
from pathlib import Path

import anndata2ri
import lamindb as ln
import rpy2.robjects

%load_ext rpy2.ipython

assert ln.setup.settings.instance.slug == "theislab/sc-best-practices"

ln.track()

→ connected lamindb: theislab/sc-best-practices
→ created Transform('QaILJZMpZyJ40000'), started new Run('1YEGroVZ...') at 2025-03-26 12:58:46 UTC


## 명명법

여러 언어에 대해 이야기하는 것이 혼란스러울 수 있으므로 다음과 같은 규칙을 사용하려고 합니다:

- **{package}** - R 패키지
- `package::function()` - R 패키지의 함수
- **package** - Python 패키지
- `package.function()` - Python 패키지의 함수
- **강조** - 기타 중요한 개념
- `code` - 객체, 변수 등 코드의 다른 부분. 파일이나 디렉토리에도 사용됩니다.

## 디스크 기반 상호운용성

언어 간 이동의 첫 번째 접근 방식은 디스크 기반 상호운용성입니다.
이는 한 언어에서 파일을 디스크에 쓰고 다른 언어에서 해당 파일을 읽는 것을 포함합니다.
많은 경우 이 접근 방식은 아래에서 설명할 인메모리 상호운용성보다 간단하고 신뢰할 수 있으며 확장 가능합니다.
하지만 저장 공간 요구 사항이 커지고 상호 작용성이 감소하는 단점이 있습니다.
디스크 기반 상호운용성은 분석의 각 단계에 대한 프로세스가 확립되어 있고 한 객체에서 다음 객체로 전달하고자 할 때 특히 잘 작동합니다(특히 [Nextflow](https://www.nextflow.io/index.html) 또는 [snakemake](https://snakemake.readthedocs.io/en/stable/)와 같은 워크플로우 관리자를 사용하여 개발된 파이프라인의 일부로).
그러나 디스크 기반 상호운용성은 데이터 탐색이나 방법 실험과 같은 대화형 단계에는 덜 편리합니다. 언어 간에 이동할 때마다 새 파일을 작성해야 하기 때문입니다.

(introduction-interoperability-key-takeaway-2)=
### 간단한 형식

단일 세포 데이터용으로 특별히 개발된 파일 형식을 논의하기 전에, CSV, TSV, JSON 등과 같은 일반적인 간단한 텍스트 파일 형식이 언어 간 데이터 전송에 대한 해답이 될 수 있음을 간략하게 언급하고 싶습니다.
이러한 형식은 일부 분석이 수행되었고 실험에 대한 정보의 일부를 전송하려는 경우에 잘 작동합니다.
예를 들어, 세포 메타데이터만 전송하고 특징 메타데이터, 발현 행렬 등은 필요하지 않을 수 있습니다.
간단한 텍스트 형식을 사용하는 장점은 거의 모든 언어에서 잘 지원되며 단일 세포 관련 패키지가 필요하지 않다는 것입니다.
그러나 전송하려는 내용이 복잡해지면 비실용적이 될 수 있습니다.

### HDF5 기반 형식

[계층적 데이터 형식 버전 5](https://www.hdfgroup.org/solutions/hdf5/) (HDF5)는 단일 세포 데이터를 저장하는 데 가장 일반적인 오픈 소스 파일 형식입니다.
이 형식은 컴퓨터의 파일 시스템과 유사한 디렉토리와 같은 구조를 사용하여 크고 복잡하며 이질적인 데이터셋을 위해 설계되었습니다.
이를 통해 여러 유형의 데이터를 단일 파일 내에 체계적인 계층 구조로 저장할 수 있습니다. HDF5는 매우 유연하지만, 이와 상호 작용하려면 파일 내에서 데이터가 어떻게 구조화되어 있는지에 대한 지식이 필요합니다.
이 프로세스를 표준화하기 위해 HDF5 파일에 단일 세포 데이터를 저장하기 위한 특정 지침이 개발되었습니다.

#### H5AD

H5AD 형식은 scverse 패키지에서 사용하는 `AnnData` 객체의 HDF5 디스크 표현이며 일반적으로 단일 세포 데이터셋을 공유하는 데 사용됩니다.
scverse 생태계의 일부이므로 Python에서 이러한 파일을 읽고 쓰는 것은 잘 지원되며 [**anndata** 패키지](https://anndata.readthedocs.io/en/latest/index.html)의 핵심 기능의 일부입니다(형식에 대한 자세한 내용은 [여기](https://anndata.readthedocs.io/en/latest/fileformat-prose.html) 참조).

상호운용성을 시연하기 위해, 표준 분석 워크플로우의 일부 단계를 거쳐 다양한 슬롯을 채운 작고 무작위로 생성된 데이터셋을 로드할 것입니다.

In [3]:
af = ln.Artifact.get(key="introduction/interoperability_adata.h5ad", is_latest=True)
adata = af.load()
adata

AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'

이 모의 객체를 H5AD 파일로 디스크에 써서 R에서 해당 파일을 어떻게 읽을 수 있는지 시연할 것입니다.

In [4]:
temp_dir = tempfile.TemporaryDirectory()
h5ad_file = str(Path(temp_dir.name) / "example.h5ad")

adata.write_h5ad(h5ad_file)

여러 R 패키지가 H5AD 파일 읽기 및 쓰기를 지원합니다.
그러나 일반적으로 R과 Python 간의 데이터 브리징을 위해 인메모리 변환 단계를 사용하여 파일 처리를 위해 Python **anndata** 패키지를 래핑합니다.

##### Bioconductor로 H5AD 읽기/쓰기

[Bioconductor **{zellkonverter}** 패키지](https://bioconductor.org/packages/zellkonverter/)는 호환되는 Python 환경을 관리하기 위해 [**{basilisk}** 패키지](https://bioconductor.org/packages/basilisk/)를 사용하여 H5AD 파일 처리를 단순화합니다.
즉, Bioconductor 사용자가 Python 지식 없이도 H5AD 파일을 원활하게 읽고 쓸 수 있도록 합니다.

불행히도 이 책이 만들어지는 방식 때문에 여기에서 코드를 직접 실행할 수 없습니다. 대신 R 세션에서 실행될 때의 코드와 출력 모양을 보여드리겠습니다:

```r
sce <- zellkonverter::readH5AD(h5ad_file, verbose = TRUE)
```

```
ℹ Using the Python reader
ℹ Using anndata version 0.8.0
✔ Read /.../luke.zappia/Downloads/example.h5ad [113ms]
✔ uns$hvg$flavor converted [17ms]
✔ uns$hvg converted [50ms]
✔ uns$log1p converted [25ms]
✔ uns$neighbors converted [18ms]
✔ uns$pca$params$use_highly_variable converted [16ms]
✔ uns$pca$params$zero_center converted [16ms]
✔ uns$pca$params converted [80ms]
✔ uns$pca$variance converted [17ms]
✔ uns$pca$variance_ratio converted [16ms]
✔ uns$pca converted [184ms]
✔ uns$umap$params$a converted [16ms]
✔ uns$umap$params$b converted [16ms]
✔ uns$umap$params converted [80ms]
✔ uns$umap converted [112ms]
✔ uns converted [490ms]
✔ Converting uns to metadata ... done
✔ X matrix converted to assay [29ms]
✔ layers$counts converted [27ms]
✔ Converting layers to assays ... done
✔ var converted to rowData [25ms]
✔ obs converted to colData [24ms]
✔ varm$PCs converted [18ms]
✔ varm converted [47ms]
✔ Converting varm to rowData$varm ... done
✔ obsm$X_pca converted [15ms]
✔ obsm$X_umap converted [16ms]
✔ obsm converted [80ms]
✔ Converting obsm to reducedDims ... done
ℹ varp is empty and was skipped
✔ obsp$connectivities converted [22ms]
✔ obsp$distances converted [23ms]
✔ obsp converted [92ms]
✔ Converting obsp to colPairs ... done
✔ SingleCellExperiment constructed [164ms]
ℹ Skipping conversion of raw
✔ Converting AnnData to SingleCellExperiment ... done
```

상세 출력을 켰기 때문에 **{zellkonverter}**가 Python을 사용하여 파일을 읽고 `AnnData` 객체의 각 부분을 Bioconductor `SingleCellExperiment` 객체로 변환하는 방법을 볼 수 있습니다. 결과가 어떻게 보이는지 확인할 수 있습니다:

```r
sce
```

```
class: SingleCellExperiment
dim: 2000 100
metadata(5): hvg log1p neighbors pca umap
assays(2): X counts
rownames(2000): Gene_0 Gene_1 ... Gene_1998 Gene_1999
rowData names(11): n_cells_by_counts mean_counts ... dispersions_norm
  varm
colnames(100): Cell_0 Cell_1 ... Cell_98 Cell_99
colData names(8): n_genes_by_counts log1p_n_genes_by_counts ...
  pct_counts_in_top_200_genes pct_counts_in_top_500_genes
reducedDimNames(2): X_pca X_umap
mainExpName: NULL
altExpNames(0):
```

이 객체는 Bioconductor 패키지에서 정상적으로 사용할 수 있습니다. 새 H5AD 파일을 쓰려면 `writeH5AD()` 함수를 사용할 수 있습니다:

```r
zellkonverter_h5ad_file <- tempfile(fileext = ".h5ad")
zellkonverter::writeH5AD(sce, zellkonverter_h5ad_file, verbose = TRUE)
```

```
ℹ Using anndata version 0.8.0
ℹ Using the 'X' assay as the X matrix
✔ Selected X matrix [29ms]
✔ assays$X converted to X matrix [50ms]
✔ additional assays converted to layers [30ms]
✔ rowData$varm converted to varm [28ms]
✔ reducedDims converted to obsm [68ms]
✔ metadata converted to uns [24ms]
ℹ rowPairs is empty and was skipped
✔ Converting AnnData to SingleCellExperiment ... done
✔ Wrote '/.../.../rj/.../T/.../file102cfa97cc51.h5ad ' [133ms]
```

그런 다음 Python에서 이 파일을 읽을 수 있습니다:

```python
scanpy.read_h5ad(zellkonverter_h5ad_file)
```

```
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'X_name', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```

**{zellkonverter}** 함수를 처음 실행하는 경우 특별한 Conda 환경을 생성하며 시간이 걸릴 수 있습니다.
생성된 후에는 이 환경이 후속 함수 호출에 재사용됩니다.
**{zellkonverter}**는 객체의 일부를 선택적으로 읽거나 쓰는 것과 같은 옵션도 제공합니다.
자세한 내용은 패키지 설명서를 참조하십시오.

`SingleCellExperiment` 객체를 H5AD 파일로 쓰는 유사한 기능은 [**{sceasy}** 패키지](https://github.com/cellgeni/sceasy)에서 사용할 수 있습니다.
이러한 패키지는 효과적이지만 Python을 래핑하면 약간의 오버헤드가 발생하며, 이는 향후 네이티브 R H5AD 작성기/읽기 프로그램이 최적화하는 데 도움이 될 수 있습니다.

##### **{Seurat}**으로 H5AD 읽기/쓰기

`h5ad_file`은 `Path` 객체이지만, 이 노트북에서 R을 사용하는 방식은 문자열을 기대하므로 `string` 객체로 변환합니다.

In [5]:
h5ad_file = str(h5ad_file)

`Seurat` 객체와 H5AD 파일 간의 변환은 [이 튜토리얼에서 제안한 대로](https://mojaveazure.github.io/seurat-disk/articles/convert-anndata.html) 2단계 프로세스입니다.
먼저, H5AD 파일은 [**{SeuratDisk}** 패키지](https://mojaveazure.github.io/seurat-disk/)를 사용하여 `Seurat` 객체를 위한 사용자 정의 HDF5 형식인 H5Seurat 파일로 변환됩니다.
그런 다음 H5Seurat 파일은 `Seurat` 객체로 읽힙니다.

In [6]:
%%R -i h5ad_file

message("Converting H5AD to H5Seurat...")
SeuratDisk::Convert(h5ad_file, dest = "h5seurat", overwrite = TRUE)
message("Reading H5Seurat...")
h5seurat_file <- gsub(".h5ad", ".h5seurat", h5ad_file)
seurat <- SeuratDisk::LoadH5Seurat(h5seurat_file, assays = "RNA")
message("Read Seurat object:")
seurat


R[write to console]: Converting H5AD to H5Seurat...




    an issue that caused a segfault when used with rpy2:
    https://github.com/rstudio/reticulate/pull/1188
    Make sure that you use a version of that package that includes
    the fix.
    

R[write to console]: Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

R[write to console]: 경고:
R[write to console]:  Unknown file type: h5ad

R[write to console]: 경고:
R[write to console]:  'assay' not set, setting to 'RNA'

R[write to console]: Creating h5Seurat file for version 3.1.5.9900

R[write to console]: Adding X as data

R[write to console]: Adding X as counts

R[write to console]: Adding meta.features from var

R[write to console]: Adding X_pca as cell embeddings for pca

R[write to console]: Adding X_umap as cell embeddings for umap

R[write to console]: Adding PCs as feature loadings fpr pca

R[write to console]: Adding miscellaneous information for pca

R[write to console]: Adding standard deviations for pca

R[write to console]: Adding miscellaneous information for umap

R[write to console]: Adding hvg to miscellaneous data

R[write to console]: Adding log1p to miscellaneous data

R[write to console]: Adding layer cou

An object of class Seurat 
2000 features across 100 samples within 1 assay 
Active assay: RNA (2000 features, 0 variable features)
 2 layers present: counts, data
 2 dimensional reductions calculated: pca, umap


`Seurat` 객체 변환은 구조적 차이로 인해 `AnnData`나 `SingleCellExperiment`에 비해 더 복잡합니다.
자세한 내용은 [변환 함수 설명서](https://mojaveazure.github.io/seurat-disk/reference/Convert.html)를 참조하십시오.

**{sceasy}** 패키지를 사용하면 H5AD 파일을 `Seurat` 또는 `SingleCellExperiment` 객체로 직접 읽을 수 있습니다.
전용 Python 환경에 의존하는 **{zellkonverter}**와 달리, **{sceasy}**는 특별한 설정 없이 Python 함수를 래핑합니다.
그러나 이는 환경을 수동으로 구성하고, R이 해당 환경을 찾을 수 있도록 하고, 필요한 패키지를 설치해야 함을 의미합니다.

```r
sceasy_seurat <- sceasy::convertFormat(h5ad_file, from="anndata", to="seurat")
sceasy_seurat
```
```
Warning: Feature names cannot have underscores ('_'), replacing with dashes ('-')
X -> counts
An object of class Seurat
2000 features across 100 samples within 1 assay
Active assay: RNA (2000 features, 0 variable features)
 2 dimensional reductions calculated: pca, umap
```

##### **{anndata}**로 H5AD 읽기/쓰기

R [**{anndata}** 패키지](https://anndata.dynverse.org/index.html)도 H5AD 파일을 읽는 데 사용할 수 있습니다.
그러나 위 패키지와 달리 네이티브 R 객체로 변환하지 않습니다.
대신 Python 객체에 대한 R 인터페이스를 제공합니다.
이는 데이터에 접근하는 데 유용하지만, 이 입력을 받아들이는 분석 패키지는 거의 없으므로 일반적으로 추가적인 인메모리 변환이 필요합니다.

#### Loom

[Loom 파일 형식](http://loompy.org/)은 omics 데이터를 위한 오래된 HDF5 기반 사양입니다.
`AnnData` 및 `SingleCellExperiment`와 구조적으로 유사하지만 H5AD처럼 특정 분석 생태계에 묶여 있지는 않습니다.
Loom 형식 지원은 [R](https://github.com/mojaveazure/loomR)과 [Python](https://pypi.org/project/loompy/) 모두에서 가능하며, Loom 파일을 쓰기 위한 [Bioconductor 패키지](https://bioconductor.org/packages/LoomExperiment/)를 통해서도 가능합니다.
그러나 핵심 생태계 패키지에서 제공하는 상위 수준 인터페이스를 사용하는 것이 더 편리한 경우가 많습니다.
데이터셋 공유 외에도, Loom 파일은 {ref}`RNA 속도 분석 <trajectories:rna-velocity>`을 위해 [velocyto](http://velocyto.org/)를 사용하여 접합 및 미접합 판독값을 분석할 때 흔히 접하게 됩니다.

### RDS 파일

단일 세포 데이터셋을 공유하는 데 사용되는 또 다른 파일 형식은 RDS 형식입니다.
이것은 임의의 `R` 객체를 직렬화하는 데 사용되는 이진 형식입니다(Python Pickle 파일과 유사).
`SingleCellExperiment`와 `Seurat` 객체가 항상 일치하는 디스크 상 표현을 갖지 않았기 때문에, RDS 파일은 R 분석 결과를 공유하는 데 때때로 사용됩니다.
이는 분석 프로젝트 내에서는 괜찮지만, 다른 생태계와의 상호운용성 부족으로 인해 공개적으로 또는 협력자와 데이터를 공유하는 데 사용하는 것은 권장하지 않습니다.
대신, 여러 언어에서 읽을 수 있는 위에서 언급한 HDF5 형식 중 하나를 사용하는 것이 좋습니다.

### 새로운 디스크 상 형식

HDF5 기반 형식이 현재 단일 세포 데이터의 디스크 상 표현의 표준이지만, [Zarr](https://zarr.dev/) 및 [TileDB](https://tiledb.com/)와 같은 다른 최신 기술은 특히 매우 큰 데이터셋 및 기타 양식에 대해 몇 가지 장점이 있습니다.
우리는 이러한 형식에 대한 사양이 향후 개발될 것으로 예상하며, 이는 커뮤니티에서 채택될 수 있습니다(**anndata**는 이미 Zarr 파일을 지원합니다).

```{admonition} 핵심 사항
- `SingleCellExperiment`: R의 단일 세포 데이터에 대한 Bioconductor 표준. `Anndata`와 `SingleCellExperiment` 간의 변환은 `zellkonverter`로 수행할 수 있습니다.
- `Seurat`: R의 단일 세포 분석 프레임워크. `SeuratDisk`를 통해 `AnnData`를 읽고 쓰거나 `sceasy`를 통해 변환할 수 있습니다.
- `AnnData`: Python의 단일 세포 데이터에 대한 표준 형식
```

## 인메모리 상호운용성

상호운용성의 두 번째 접근 방식은 객체의 인메모리 표현에 대해 작업하는 것입니다.
이 접근 방식은 두 프로그래밍 언어의 활성 세션을 동시에 실행하고 둘 다에서 동일한 객체에 액세스하거나 필요에 따라 변환하는 것을 포함합니다.
일반적으로 한 언어가 주 환경 역할을 하고 다른 언어에 대한 인터페이스가 있습니다.
이는 분석가가 두 언어에서 동시에 작업할 수 있도록 하므로 대화형 분석에 매우 유용할 수 있습니다.
또한 여러 언어를 사용하는 문서(예: 이 책)를 만들 때 자주 사용됩니다.
그러나 인메모리 상호운용성에는 몇 가지 단점이 있습니다.
분석가는 두 환경을 설정하고 사용하는 데 익숙해야 하며, 복잡한 객체는 언어 간에 완전히 지원되지 않을 수 있으며, 데이터 중복은 메모리 오버헤드를 증가시켜 대용량 데이터셋에는 덜 적합합니다.

### R 생태계 간 상호운용성

R과 Python 간의 인메모리 상호운용성을 살펴보기 전에, 두 R 생태계 간 변환이라는 더 간단한 경우를 고려해 보겠습니다.
**{Seurat}** 패키지는 [이 비네트에서 설명한 대로](https://satijalab.org/seurat/articles/conversion_vignette.html) 이 변환을 수행하는 함수를 제공합니다.

In [7]:
%%R
sce_from_seurat <- Seurat::as.SingleCellExperiment(seurat)
sce_from_seurat


class: SingleCellExperiment 
dim: 2000 100 
metadata(0):
assays(2): counts logcounts
rownames(2000): Gene-0 Gene-1 ... Gene-1998 Gene-1999
rowData names(0):
colnames(100): Cell_0 Cell_1 ... Cell_98 Cell_99
colData names(9): n_genes_by_counts log1p_n_genes_by_counts ...
  pct_counts_in_top_500_genes ident
reducedDimNames(2): PCA UMAP
mainExpName: RNA
altExpNames(0):


In [8]:
%%R
seurat_from_sce <- Seurat::as.Seurat(sce_from_seurat)
seurat_from_sce


An object of class Seurat 
2000 features across 100 samples within 1 assay 
Active assay: RNA (2000 features, 0 variable features)
 2 layers present: counts, data
 2 dimensional reductions calculated: PCA, UMAP


여기서 어려운 부분은 두 객체의 구조 차이 때문입니다.
변환 함수가 어떤 정보를 변환하고 어디에 배치할지 알 수 있도록 인수가 올바르게 설정되었는지 확인하는 것이 중요합니다.

많은 경우 `Seurat` 객체를 `SingleCellExperiment`로 변환할 필요가 없을 수 있습니다.
이는 단일 세포 분석을 위한 많은 핵심 Bioconductor 패키지가 입력으로 행렬을 허용하도록 설계되었기 때문입니다.

In [9]:
%%R
# Calculate Counts Per Million using the Bioconductor scuttle package
# with a matrix in a Seurat object
cpm <- scuttle::calculateCPM(Seurat::GetAssayData(seurat, slot = "counts"))
cpm[1:10, 1:10]


10 x 10 sparse Matrix of class "dgCMatrix"


R[write to console]:   [[ suppressing 10 column names ‘Cell_0’, ‘Cell_1’, ‘Cell_2’ ... ]]



                                                                      
Gene-0 594.0456    .     984.0238  610.329 964.4394 616.4169    .     
Gene-1 594.0456 1168.519 622.6029    .     608.7911   .         .     
Gene-2   .         .     984.0238    .     608.7911 973.2674 1205.5970
Gene-3 594.0456    .     622.6029  610.329 964.4394 616.4169    .     
Gene-4 594.0456    .     622.6029  966.820 964.4394   .         .     
Gene-5 594.0456  580.157 622.6029  610.329 608.7911   .       601.7422
Gene-6   .         .     622.6029  610.329   .      616.4169  601.7422
Gene-7 594.0456  580.157 622.6029  966.820 608.7911 616.4169    .     
Gene-8 594.0456  580.157   .      1219.608 608.7911 973.2674    .     
Gene-9 942.9097    .       .         .     964.4394   .       954.8014
                                    
Gene-0  576.0808  619.7236  604.3937
Gene-1  918.4111 1234.7593  958.4625
Gene-2    .       619.7236 1209.8233
Gene-3    .       619.7236  958.4625
Gene-4  576.0808  979.8761  604.39

그러나 올바른 정보에 액세스하고 필요한 경우 결과를 올바른 위치에 저장하고 있는지 확인하는 것이 중요합니다.

### Python에서 R에 액세스하기

Python에서 R로의 인터페이스는 [**rpy2** 패키지](https://rpy2.github.io/doc/latest/html/index.html)에 의해 제공됩니다.
이를 통해 Python에서 R 함수 및 객체에 액세스할 수 있습니다.
예를 들어:

In [10]:
counts_mat = adata.layers["counts"].T.toarray()

with rpy2.robjects.conversion.localconverter(rpy2.robjects.numpy2ri.converter):
    rpy2.robjects.globalenv["counts_mat"] = counts_mat

cpm = rpy2.robjects.r("scuttle::calculateCPM(counts_mat)")
cpm

494.804552,494.804552,0.000000,...,519.750520,519.750520,0.000000


일반적인 Python 객체(리스트, 행렬, `DataFrame` 등)도 R로 전달할 수 있습니다.

Jupyter 노트북을 사용하고 있다면(이 책에서처럼) IPython 매직 인터페이스를 사용하여 네이티브 R 코드로 셀을 만들 수 있습니다(필요에 따라 객체 전달).
예를 들어, 셀을 `%%R -i input -o output`으로 시작하면 `input`을 입력으로 받아 R 코드를 실행한 다음 `output`을 출력으로 반환하라는 의미입니다.

In [11]:
%%R -i counts_mat -o magic_cpm
# R code running using IPython magic
magic_cpm <- scuttle::calculateCPM(counts_mat)


In [12]:
# Python code accessing the results
magic_cpm

array([[ 494.8045522 ,    0.        , 1027.2213662 , ...,    0.        ,
           0.        ,  519.75051975],
       [ 494.8045522 , 1445.78313253,  513.6106831 , ...,    0.        ,
         499.5004995 ,  519.75051975],
       [   0.        ,    0.        , 1027.2213662 , ...,  485.90864917,
         499.5004995 ,    0.        ],
       ...,
       [ 494.8045522 ,    0.        ,  513.6106831 , ...,    0.        ,
           0.        ,  519.75051975],
       [ 989.6091044 ,  481.92771084,    0.        , ...,    0.        ,
         499.5004995 ,  519.75051975],
       [2474.02276101,  963.85542169,  513.6106831 , ...,  485.90864917,
         999.000999  ,    0.        ]])

이것이 이후 장에서 가장 흔하게 보게 될 접근 방식입니다.
**rpy2** 사용에 대한 자세한 내용은 [설명서](https://rpy2.github.io/doc/latest/html/index.html)를 참조하십시오.

이러한 방식으로 단일 세포 데이터를 사용하려면 [**anndata2ri** 패키지](https://icb-anndata2ri.readthedocs-hosted.com/en/latest/)가 특히 유용합니다.
**rpy2**의 확장 기능으로, R이 `AnnData` 객체를 `SingleCellExperiment` 객체로 인식하도록 하여 불필요한 변환을 제거하고 Python 객체에서 R 코드를 원활하게 실행할 수 있도록 합니다.
또한 희소 **scipy** 행렬의 변환을 용이하게 합니다.

이 예에서 Python 세션의 `AnnData` 객체를 R로 전달하려면 먼저 `SingleCellExperiment`로 변환해야 합니다.

In [13]:
with rpy2.robjects.conversion.localconverter(anndata2ri.converter):
    r_adata = rpy2.robjects.conversion.py2rpy(adata)

이제 R로 전달할 수 있습니다.

In [14]:
%%R -i r_adata
qc <- scuttle::perCellQCMetrics(r_adata)
head(qc)

DataFrame with 6 rows and 3 columns
             sum  detected     total
       <numeric> <integer> <numeric>
Cell_0      2021      1297      2021
Cell_1      2075      1314      2075
Cell_2      1947      1233      1947
Cell_3      1986      1250      1986
Cell_4      1987      1255      1987
Cell_5      1930      1266      1930


객체(또는 그 일부)가 올바르게 인터페이스될 수 없는 경우(예: 지원되지 않는 데이터 유형이 있는 경우) 여전히 문제가 발생할 수 있습니다.
이 경우 액세스하기 전에 객체를 수정해야 할 수 있습니다.

### R에서 Python에 액세스하기

R 세션에서 Python에 액세스하는 것은 Python에서 R에 액세스하는 것과 유사하지만 여기서는 인터페이스가 [**{reticulate}** 패키지](https://rstudio.github.io/reticulate/)에 의해 제공됩니다.
로드되면 R에서 Python 함수 및 객체에 액세스할 수 있습니다.

In [15]:
%%R
reticulate_list <- reticulate::r_to_py(LETTERS)
print(reticulate_list)
py_builtins <- reticulate::import_builtins()
py_builtins$zip(letters, LETTERS)


List (26 items)


[RMarkdown](https://rmarkdown.rstudio.com/) 또는 [Quarto](https://quarto.org/) 문서에서 작업하는 경우 **{reticulate}** Python 엔진을 사용하여 네이티브 Python 청크를 작성할 수도 있습니다.
이렇게 하면 매직 `r` 및 `py` 변수를 사용하여 다른 언어의 객체에 액세스할 수 있습니다(다음 코드는 실행되지 않는 예제입니다).

````
```{r}
# Python 객체에 액세스하는 R 청크
print(py$py_object)
```

```{python}
# R 객체에 액세스하는 Python 청크
print(r$r_object)
```
````

**anndata2ri**와 달리, Python이 `SingleCellExperiment` 또는 `Seurat` 객체를 `AnnData` 객체로 볼 수 있도록 하는 직접적인 인터페이스를 제공하는 R 패키지는 없습니다.
그러나 **{reticulate}**를 사용하여 `AnnData`의 대부분의 부분에 계속 액세스할 수 있습니다(이 코드는 실행되지 않음).

```r
# Python 환경에서 AnnData 객체 인쇄
py$adata
```
```
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```
```r
# 또는 Python anndata 패키지를 사용하여 H5AD 파일 읽기
anndata <- reticulate::import("anndata")
anndata$read_h5ad(h5ad_file)
```
```
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```
```r
# obs 슬롯에 액세스, pandas DataFrame은 자동으로 R data.frame으로 변환됩니다.
head(adata$obs)
```
```
       n_genes_by_counts log1p_n_genes_by_counts total_counts
Cell_0              1246                7.128496         1965
Cell_1              1262                7.141245         2006
Cell_2              1262                7.141245         1958
Cell_3              1240                7.123673         1960
Cell_4              1296                7.167809         2027
Cell_5              1231                7.116394         1898
       log1p_total_counts pct_counts_in_top_50_genes
Cell_0           7.583756                  10.025445
Cell_1           7.604396                   9.521436
Cell_2           7.580189                   9.959142
Cell_3           7.581210                   9.183673
Cell_4           7.614805                   9.718796
Cell_5           7.549083                  10.168599
       pct_counts_in_top_100_genes pct_counts_in_top_200_genes
Cell_0                    17.65903                    30.89059
Cell_1                    16.99900                    29.71087
Cell_2                    17.62002                    30.28601
Cell_3                    16.83673                    30.45918
Cell_4                    17.11889                    30.04440
Cell_5                    18.07165                    30.29505
       pct_counts_in_top_500_genes
Cell_0                    61.42494
Cell_1                    59.62114
Cell_2                    60.92952
Cell_3                    61.07143
Cell_4                    59.64480
Cell_5                    61.48577
```

위에서 언급했듯이 R **{anndata}** 패키지는 `AnnData` 객체에 대한 R 인터페이스를 제공하지만 현재 많은 분석 패키지에서 사용되지 않습니다.

전체 객체로 작업해야 하는 더 복잡한 분석의 경우 객체를 R과 Python 간에 완전히 변환해야 할 수 있습니다.
 이 접근 방식은 데이터 중복으로 인해 메모리 효율적이지는 않지만 더 넓은 범위의 패키지에 액세스할 수 있습니다.

**{zellkonverter}** 패키지는 이 변환을 위한 함수를 제공합니다.
H5AD 파일을 읽는 함수와 달리 이 프로세스는 특별히 생성된 환경이 아닌 표준 Python 환경을 사용합니다(코드는 실행되지 않음).

```r
# AnnData를 SingleCellExperiment로 변환
sce <- zellkonverter::AnnData2SCE(adata, verbose = TRUE)
sce
```
```
✔ uns$hvg$flavor converted [21ms]
✔ uns$hvg converted [62ms]
✔ uns$log1p converted [22ms]
✔ uns$neighbors converted [21ms]
✔ uns$pca$params$use_highly_variable converted [22ms]
✔ uns$pca$params$zero_center converted [31ms]
✔ uns$pca$params converted [118ms]
✔ uns$pca$variance converted [17ms]
✔ uns$pca$variance_ratio converted [17ms]
✔ uns$pca converted [224ms]
✔ uns$umap$params$a converted [15ms]
✔ uns$umap$params$b converted [17ms]
✔ uns$umap$params converted [80ms]
✔ uns$umap converted [115ms]
✔ uns converted [582ms]
✔ Converting uns to metadata ... done
✔ X matrix converted to assay [44ms]
✔ layers$counts converted [29ms]
✔ Converting layers to assays ... done
✔ var converted to rowData [37ms]
✔ obs converted to colData [23ms]
✔ varm$PCs converted [18ms]
✔ varm converted [49ms]
✔ Converting varm to rowData$varm ... done
✔ obsm$X_pca converted [17ms]
✔ obsm$X_umap converted [17ms]
✔ obsm converted [80ms]
✔ Converting obsm to reducedDims ... done
ℹ varp is empty and was skipped
✔ obsp$connectivities converted [21ms]
✔ obsp$distances converted [22ms]
✔ obsp converted [89ms]
✔ Converting obsp to colPairs ... done
✔ SingleCellExperiment constructed [241ms]
ℹ Skipping conversion of raw
✔ Converting AnnData to SingleCellExperiment ... done
class: SingleCellExperiment
dim: 2000 100
metadata(5): hvg log1p neighbors pca umap
assays(2): X counts
rownames(2000): Gene_0 Gene_1 ... Gene_1998 Gene_1999
rowData names(11): n_cells_by_counts mean_counts ... dispersions_norm
  varm
colnames(100): Cell_0 Cell_1 ... Cell_98 Cell_99
colData names(8): n_genes_by_counts log1p_n_genes_by_counts ...
  pct_counts_in_top_200_genes pct_counts_in_top_500_genes
reducedDimNames(2): X_pca X_umap
mainExpName: NULL
altExpNames(0):
```

반대로도 동일하게 수행할 수 있습니다:

```r
adata2 <- zellkonverter::SCE2AnnData(sce, verbose = TRUE)
adata2
```
```
ℹ Using the 'X' assay as the X matrix
✔ Selected X matrix [27ms]
✔ assays$X converted to X matrix [38ms]
✔ additional assays converted to layers [31ms]
✔ rowData$varm converted to varm [15ms]
✔ reducedDims converted to obsm [63ms]
✔ metadata converted to uns [23ms]
ℹ rowPairs is empty and was skipped
✔ Converting AnnData to SingleCellExperiment ... done
AnnData object with n_obs × n_vars = 100 × 2000
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'X_name', 'hvg', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'counts'
    obsp: 'connectivities', 'distances'
```

## 다중 모드 데이터의 상호운용성

다중 모드 데이터의 복잡성은 상호운용성에 추가적인 과제를 제시합니다.
`SingleCellExperiment`("대체 실험"을 통해, 세포에 대해 동일한 열 차원을 공유해야 함)와 `Seurat`("분석" 사용) 모두 여러 양식을 지원합니다.
그러나 `AnnData`는 단일 모드 데이터로 제한됩니다.

이러한 한계를 해결하기 위해, 다중 모드 데이터셋을 위한 `AnnData`의 확장으로 개발된 `MuData` 객체([분석 프레임워크 및 도구 장]({ref}`분석 프레임워크 및 도구 장 <introduction:analysis-frameworks>`)에서 소개됨)가 있습니다.
개발자들은 설계에 상호운용성을 고려했습니다.
MuData의 주요 플랫폼은 Python이지만, 저자들은 H5MU 형식의 디스크 상 파일을 `Seurat` 객체로 읽기 위한 [MuDataSeurat R 패키지](https://pmbio.github.io/MuDataSeurat/)와 Bioconductor `MultiAssayExperiment` 객체로 동일한 작업을 수행하기 위한 [MuData R 패키지](https://bioconductor.org/packages/MuData/)를 제공했습니다. 이 공식 지원은 매우 유용하지만 객체 간의 차이로 인해 여전히 일부 불일치가 있습니다. MuData 저자들은 또한 `AnnData`와 `MuData`의 [Julia 구현](https://docs.juliahub.com/Muon/QfqCh/0.1.1/objects/)을 제공합니다.

아래는 Python 및 R 패키지를 사용하여 작은 예제 `MuData` 데이터셋을 읽고 쓰는 예제입니다.

이를 해결하기 위해, 다중 모드 데이터셋을 위한 `AnnData`의 확장인 `MuData` 객체([분석 프레임워크 및 도구 장]({ref}`분석 프레임워크 및 도구 장 <introduction:analysis-frameworks>`)에서 소개됨)가 있습니다.
상호운용성을 염두에 두고 설계된 `MuData`는 주로 Python 기반 프레임워크이지만, 저자들은 [MuDataSeurat R 패키지](https://pmbio.github.io/MuDataSeurat/)를 제공했습니다.
이를 통해 H5MU 형식의 디스크 상 파일을 `Seurat` 객체로 읽을 수 있으며, [MuData R 패키지](https://bioconductor.org/packages/MuData/)를 사용하면 `MultiAssayExperiment` 객체로 변환할 수 있습니다.
이 공식 지원은 매우 유용하지만 객체 간의 차이로 인해 여전히 일부 불일치가 있습니다.
`MuData` 저자들은 또한 `AnnData`와 `MuData`의 [Julia 구현](https://docs.juliahub.com/Muon/QfqCh/0.1.1/objects/)을 제공합니다.

아래는 Python 및 R 패키지를 사용하여 작은 예제 `MuData` 데이터셋을 읽고 쓰는 예제입니다.

### 파이썬

In [29]:
# Read file
af_mudata = ln.Artifact.get(
    key="introduction/interoperability_mdata.h5mu", is_latest=True
)
mdata = af_mudata.load()
print(mdata)

/Users/seohyon/miniconda3/envs/interoperability/lib/python3.12/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/Users/seohyon/miniconda3/envs/interoperability/lib/python3.12/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


MuData object with n_obs × n_vars = 1000 × 150
  var:	'dummy_var'
  2 modalities
    A:	1000 x 100
      obs:	'dummy_obs'
      var:	'dummy_var'
    B:	1000 x 50
      obs:	'dummy_obs'
      var:	'dummy_var'


### R

#### Bioconductor

`MultiAssayExperiment` 객체에서 읽기/쓰기

In [36]:
import shutil
from pathlib import Path

# Save MuData file locally for R
Path("data").mkdir(parents=True, exist_ok=True)

local_path = af_mudata.cache().path

target_path = Path("data") / "interoperability_mdata.h5mu"
shutil.copy(local_path, target_path)
print(f"File copied to: {target_path}")

File copied to: data/interoperability_mdata.h5mu


In [37]:
%%R
mae <- MuData::readH5MU("data/interoperability_mdata.h5mu")
print(mae)

bioc_h5mu_file <- tempfile(fileext = ".h5mu")
MuData::writeH5MU(mae, bioc_h5mu_file)


A MultiAssayExperiment object of 2 listed
 experiments with user-defined names and respective classes.
 Containing an ExperimentList class object of length 2:
 [1] A: SingleCellExperiment with 100 rows and 1000 columns
 [2] B: SingleCellExperiment with 50 rows and 1000 columns
Functionality:
 experiments() - obtain the ExperimentList instance
 colData() - the primary/phenotype DataFrame
 sampleMap() - the sample coordination DataFrame
 `$`, `[`, `[[` - extract colData columns, subset, or experiment
 *Format() - convert into a long or wide DataFrame
 assays() - convert ExperimentList to a SimpleList of matrices
 exportClass() - save data to flat files


#### Seurat

`Seurat` 객체에서 읽기/쓰기

In [32]:
%%R
seurat <- MuDataSeurat::ReadH5MU("data/interoperability_mdata.h5mu")
print(seurat)

seurat_h5mu_file <- tempfile(fileext = ".h5mu")
MuDataSeurat::WriteH5MU(seurat, seurat_h5mu_file)


An object of class Seurat 
150 features across 1000 samples within 2 assays 
Active assay: A (100 features, 0 variable features)
 2 layers present: counts, data
 1 other assay present: B


## 다른 언어와의 상호운용성

여기서는 R과 Python 이외의 언어와 단일 세포 데이터의 상호운용성을 위한 몇 가지 리소스와 도구를 간략하게 나열합니다.

### 줄리아

- [Muon.jl](https://docs.juliahub.com/Muon/QfqCh/0.1.1/objects/)은 `AnnData` 및 `MuData` 객체의 Julia 구현과 H5AD 및 H5MU 형식에 대한 IO를 제공합니다.
- [scVI.jl](https://github.com/maren-ha/scVI.jl)은 `AnnData`의 Julia 구현과 H5AD 형식에 대한 IO를 제공합니다.

### 자바스크립트

- [Vitessce](http://vitessce.io/)는 Zarr 형식을 사용하여 저장된 `AnnData` 객체의 로더를 포함합니다.
- [kana 제품군](https://github.com/jkanche/kana)은 H5AD 파일 및 RDS 파일로 저장된 `SingleCellExperiment` 객체 읽기를 지원합니다.

### 러스트

- [anndata-rs](https://github.com/kaizhang/anndata-rs)는 AnnData의 Rust 구현과 H5AD 형식에 대한 고급 IO 지원을 제공합니다.

## 세션 정보

## 파이썬

In [33]:
import session_info

session_info.show()

/Users/seohyon/miniconda3/envs/interoperability/lib/python3.12/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


## R

In [34]:
%%R
sessioninfo::session_info()


─ Session info ───────────────────────────────────────────────────────────────
 setting  value
 version  R version 4.3.3 (2024-02-29)
 os       macOS 15.3.2
 system   x86_64, darwin13.4.0
 ui       unknown
 language (EN)
 collate  C
 ctype    UTF-8
 tz       Europe/Berlin
 date     2025-03-26
 pandoc   3.6.3 @ /Users/seohyon/miniconda3/envs/interoperability/bin/pandoc
 quarto   NA

─ Packages ───────────────────────────────────────────────────────────────────
 package              * version    date (UTC) lib source
 abind                  1.4-8      2024-09-12 [1] CRAN (R 4.3.3)
 beachmat               2.18.0     2023-10-24 [1] Bioconductor
 Biobase              * 2.62.0     2023-10-24 [1] Bioconductor
 BiocGenerics         * 0.48.1     2023-11-01 [1] Bioconductor
 BiocParallel           1.36.0     2023-10-24 [1] Bioconductor
 bit                    4.6.0      2025-03-06 [1] CRAN (R 4.3.3)
 bit64                  4.6.0-1    2025-01-16 [1] CRAN (R 4.3.3)
 bitops                 1.0-9   

## 참고문헌

```{bibliography}
:filter: docname in docnames
:labelprefix: int
```

## 기여자

다음 분들의 기여에 감사드립니다:

### 저자

* Luke Zappia
* 김서현

### 검토자

* Lukas Heumos
* Isaac Virshup
* Anastasia Litinetskaya
* Ludwig Geistlinger
* Peter Hickey